In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

#Read data from chunks_df.csv
df = pd.read_csv('/content/drive/MyDrive/INLPT_Project/csv_files/df.csv')

In [3]:
# Display the DataFrame with embeddings
df

,Unnamed: 0,text,section,article,number1,number2,number3,number4,number5,number6,CELEX number,embedding
0,0,Avis juridique important,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31999Y0209(01),[ -0.71743995 -2.661333 -0.5654966 2.04...
1,1,Special Report No 25/98 concerning operations ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31999Y0209(01),[-3.5942285e+00 -1.9066961e+00 1.9812379e+00 ...
2,2,SPECIAL REPORT No 25/98 concerning operations ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31999Y0209(01),[-2.76330781e+00 -3.82428586e-01 -1.05377293e+...
3,3,SPECIAL REPORT No 25/98 concerning operations ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31999Y0209(01),[-3.6374424 -2.093941 2.3462052 0.805499...
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31999Y0209(01),[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...
...,...,...,...,...,...,...,...,...,...,...,...,...
29602,29602,For the High Authority,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31966S0022,[-3.7505999 -2.234225 0.9126675 2.350292...
29603,29603,The President,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31966S0022,[-2.93085 -0.78436 2.562405 2.02705 ...
29604,29604,Dino DEL BO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31966S0022,[-2.2211666 -2.5205266 2.42058 -2.599566...
29605,29605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31966S0022,[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. ...


In [4]:
!pip install spacy
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 16.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
!pip install faiss-cpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 35.4 MB/s eta 0:00:00


In [7]:
import spacy

# Load spaCy model
nlp = spacy.load("en_core_web_md")

# Provide a query text
query_embedding = nlp("the Agreement on Partnership and Cooperation was signed by the European Communities and their Members States and Kazakhstan").vector


In [8]:
import numpy as np
import faiss
import spacy
from sklearn.metrics.pairwise import cosine_similarity

# Load Spacy model
nlp = spacy.load("en_core_web_md")

# Load Faiss index
index = faiss.read_index('/content/drive/MyDrive/INLPT_Project/Faiss_VectorDB_Files/faiss_index.index')

# Load embeddings array
embeddings_array = np.load('/content/drive/MyDrive/INLPT_Project/Faiss_VectorDB_Files/faiss_embeddings.npy')

# Calculate cosine similarity between the query and all embeddings in the array
similarity_scores = cosine_similarity([query_embedding], embeddings_array)[0]

# Get the indices of the sorted similarity scores
sorted_indices = np.argsort(similarity_scores)

# Get the indices of similarity scores less than 0.1 and not equal to 0.0
min_similarity_score_indices = [index for index in sorted_indices if (similarity_scores[index] < 0.1) and (similarity_scores[index] != 0.0)]

#print(min_similarity_score_indices)


#Similarity score sorted in asending order
#similarity_score_sorted = sorted(similarity_scores)
#print(min_similarity_score_sorted)

#min_similarity_score = [score for score in similarity_score_sorted if score < 0.1]
#print(similarity_score_array)

#min_similarity_index = np.argmin(similarity_scores)

#print(min_similarity_index)

#print(embeddings_array[min_similarity_index])

#print(f"   Text: {df['text'].iloc[min_similarity_index]}")


#top = np.where((similarity_scores < 0.1) & (similarity_scores != 0.0))[0]

# Get the indices of similarity scores less than 0.1 but not equal to 0.0
#top_matches_indices = np.where((similarity_scores < 0.1) & (similarity_scores != 0.0))[0]

# Get the top 10 indices in ascending order of similarity scores
#top_10_indices = np.argsort(similarity_scores[top_matches_indices])[:1]


In [9]:
# Print the text for the top similar embeddings
print("Top Similar Texts:")
seen_similarity_scores = set()

for i, index in enumerate(min_similarity_score_indices, start=1):
    similarity_score = similarity_scores[index]

    # Check if this similarity score has been seen before
    if similarity_score not in seen_similarity_scores:
        seen_similarity_scores.add(similarity_score)

        print(f"{i}. Similarity Score: {similarity_score}")
        print(f"   Text: {df['text'].iloc[index]}")
        print()


Top Similar Texts:
1. Similarity Score: -0.1782156527042389
   Text: *****

7. Similarity Score: -0.17228484153747559
   Text: TITLE I

11. Similarity Score: -0.16586369276046753
   Text: I

25. Similarity Score: -0.16026051342487335
   Text: PART I

27. Similarity Score: -0.15801677107810974
   Text: SECTION I

28. Similarity Score: -0.14932799339294434
   Text: Sirs:

31. Similarity Score: -0.14542120695114136
   Text: Date: …………

40. Similarity Score: -0.13204863667488098
   Text: For det Europæiske Atomenergifællesskab

42. Similarity Score: -0.11686472594738007
   Text: HAVE ADOPTED THIS DIRECTIVE:

46. Similarity Score: -0.11474543809890747
   Text: Mr/Ms …

47. Similarity Score: -0.10970872640609741
   Text: Ад iмя РзспублiКi Беларусь

48. Similarity Score: -0.10944001376628876
   Text: CHAPTER I

62. Similarity Score: -0.10880161821842194
   Text: Part I

63. Similarity Score: -0.10479404032230377
   Text: HAVE ADOPTED THIS DECISION:

64. Similarity Score: -0.10252790153026581
